In [1]:
import unittest
import heapq
from unittest.mock import MagicMock
from fpl_loader import FplLoader
from expected_points_calculator import ExpectedPointsCalculator
from player import Player
from team import Team, make_transfer, optimize_team

In [2]:
class TestFplLoader(unittest.TestCase):
    def test_get_static_info(self):
        static_info = FplLoader.get_static_info()
        self.assertEqual(len(static_info['events']), 38, 'Should be 38 gameweeks')
        self.assertEqual(len(static_info['teams']), 20)
        self.assertEqual(len(static_info['element_types']), 4, 'Should be 4 different player types GKP, DEF, MID, FWD')
        self.assertIn('web_name', static_info['elements'][0])
        self.assertIn('total_points', static_info['elements'][0])
        for p in static_info['elements']:
            if p['web_name'] == 'Palmer':
                self.assertEqual(362, p['id'], 'Player unique identifier should not change')
                
    def test_get_fixtures(self):
        fixtures = FplLoader.get_fixtures()
        self.assertEqual(len(fixtures), 380)
        for fixture in fixtures:
            if (fixture['event'] == 12) and (fixture['team_h'] == 1):
                self.assertEqual(fixture['team_a'], 6, 'Arsenal play Burnely in gameweek 12')
                self.assertEqual(fixture['kickoff_time'], '2023-11-11T15:00:00Z', 'The game occurs at 15:00')
        
    def test_get_fixture_info(self):
        fixture_info = FplLoader.get_fixture_info(111)
        self.assertEqual(fixture_info['event'], 12, 'Arsenal play Burnely in gameweek 12')
        self.assertEqual(fixture_info['team_h'], 1, 'Arsenal play Burnely in gameweek 12')
        self.assertEqual(fixture_info['team_a'], 6, 'Arsenal play Burnely in gameweek 12')
        self.assertEqual(fixture_info['kickoff_time'], '2023-11-11T15:00:00Z', 'The game occurs at 15:00')
        
    def test_get_fixtures_for_gameweek(self):
        fixtures = FplLoader.get_fixtures_for_gameweek(18)
        self.assertEqual(len(fixtures), 9, 'Brentford and Man City blank')
        for fixture in fixtures:
            self.assertNotIn(4, [fixture['team_h'], fixture['team_a']], 'Brentford should blank')
            self.assertNotIn(13, [fixture['team_h'], fixture['team_a']], 'Man City should blank')
        
    def test_get_team_basic_info(self):
        man_city = FplLoader.get_team_basic_info(13)
        self.assertEqual(man_city['short_name'], 'MCI')
        self.assertEqual(man_city['strength'], 5)
        arsenal = FplLoader.get_team_basic_info(1)
        self.assertEqual(arsenal['short_name'], 'ARS')
        
    def test_get_my_team_from_api(self):
        result = FplLoader.get_my_team_from_api('aledvaghela@gmail.com', 'NCP@jxy7pau-bvm3cuf', 2085446)
        self.assertIn('transfers', result)
        self.assertIn('chips', result)
        self.assertEqual(len(result['picks']), 15)
    
    def test_get_my_team_from_local(self):
        my_team = FplLoader.get_my_team_from_local('tests_my_team.json')
        self.assertEqual(len(my_team['picks']), 15)
        self.assertEqual(my_team['transfers']['limit'], 1)
        
    def test_get_next_gameweek(self):
        self.assertEqual(FplLoader.get_next_gameweek('2023-11-10 10:30:00'), 12, 'Before deadline')
        self.assertEqual(FplLoader.get_next_gameweek('2023-11-11 15:00:00'), 13, 'After deadline')
        
    def test_get_my_historical_team_from_gameweek(self):
        my_historical_team = FplLoader.get_my_historical_team_from_gameweek(11, 2085446)
        self.assertEqual(my_historical_team['entry_history']['points'], 32, 'Very poor week, only scored 32 points!')
        self.assertEqual(len(my_historical_team['picks']), 15)
        
    def test_get_player_basic_info(self):
        player_basic_info = FplLoader.get_player_basic_info(362)
        self.assertEqual(player_basic_info['web_name'], 'Palmer')
        
    def test_get_player_detailed_info(self):
        player_detailed_info = FplLoader.get_player_detailed_info(362)
        self.assertIn('fixtures', player_detailed_info)
        self.assertIn('history', player_detailed_info)
        self.assertIn('history_past', player_detailed_info)
        
    def test_get_player_historical_info_for_gameweek(self):
        palmer = FplLoader.get_player_historical_info_for_gameweek(362, 11)
        self.assertEqual(palmer[0]['total_points'], 12, 'Cole Palmer got 12 points')
        self.assertEqual(palmer[0]['expected_goals'], '0.83', '0.83 xG')
        burnley_player = FplLoader.get_player_historical_info_for_gameweek(178, 2)
        self.assertEqual(len(burnley_player), 0, 'Burnley blanked this gameweek')
        burnley_player = FplLoader.get_player_historical_info_for_gameweek(178, 3)
        self.assertEqual(len(burnley_player), 1, 'Burnley did not blank this gameweek')
        
    def test_get_player_future_info_for_gameweek(self):
        player_future_info = FplLoader.get_player_future_info_for_gameweek(365, 38)
        self.assertEqual(len(player_future_info), 1, 'Man City have one game in the final gameweek')
        self.assertEqual(player_future_info[0]['team_a'], 19, 'Man City play West Ham on the final day')
        
    def test_get_position_info(self):
        self.assertEqual(FplLoader.get_position_info(1)['singular_name_short'], 'GKP')
        self.assertEqual(FplLoader.get_position_info(2)['singular_name_short'], 'DEF')
        self.assertEqual(FplLoader.get_position_info(3)['singular_name_short'], 'MID')
        self.assertEqual(FplLoader.get_position_info(4)['singular_name_short'], 'FWD')
        

In [3]:
class TestExpectedPointsCalculator(unittest.TestCase):
    def test_get_expected_points(self):
        class MockCalculator(ExpectedPointsCalculator):
            m = {1 : 10, 2 : 20}
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return self.m[player_id] if player_id in self.m else 0
            
        mock_calculator = MockCalculator()
        self.assertEqual(mock_calculator.get_expected_points(1, 0, 0), 10)
        self.assertEqual(mock_calculator.get_expected_points(2, 0, 0), 20)
        self.assertEqual(mock_calculator.get_expected_points(0, 100, 100), 0)
        

In [4]:
class TestPlayer(unittest.TestCase):
    def setUp(self):
        d1 = {'element': 1,
              'name': 'josh',
              'position': 'FWD',
              'club': 'ARS',
              'cost': 4,
             }
        d2 = {'element': 1,
              'name': 'josh',
              'position': 'FWD',
              'club': 'ARS',
              'cost': 5,
             }
        d3 = {'element': 100,
              'name': 'josh',
              'position': 'FWD',
              'club': 'ARS',
              'cost': 4,
             }
        
        self.p1 = Player(**d1)
        self.p2 = Player(**d2)
        self.p3 = Player(**d3)
    
    def test_player(self):
        self.assertEqual(self.p1, self.p2)
        self.assertNotEqual(self.p1, self.p3)
        

In [2]:
class MockPlayer:
    @staticmethod
    def create(element: int, position: int, club: int, cost: int):
        '''
        Helper method to create a mock player object
        :param position: int position 1 means GKP, 2 means DEF, 3 means MID, 4 means FWD
        :param club: int identifier between 1 and 20
        :param cost: cost in m times 10
        '''
        player = Player(element=element, 
                        name=str(element), 
                        position=position, 
                        club=club, 
                        cost=cost)
        
        return player

class TestTeam(unittest.TestCase):
    def setUp(self):
        # GKPs
        self.gkp_club_1 = MockPlayer.create(element=1, position=1, club=1, cost=40)
        self.gkp_club_2 = MockPlayer.create(element=2, position=1, club=2, cost=45)
        # DEFs
        self.def_club_1 = MockPlayer.create(element=3, position=2, club=1, cost=40)
        self.def_club_2 = MockPlayer.create(element=4, position=2, club=2, cost=45)
        self.def_club_3 = MockPlayer.create(element=5, position=2, club=3, cost=50)
        self.def_club_4 = MockPlayer.create(element=6, position=2, club=4, cost=55)
        self.def_club_5 = MockPlayer.create(element=7, position=2, club=5, cost=60)
        # MIDs
        self.mid_club_1 = MockPlayer.create(element=8, position=3, club=1, cost=45)
        self.mid_club_2 = MockPlayer.create(element=9, position=3, club=2, cost=50)
        self.mid_club_3 = MockPlayer.create(element=10, position=3, club=3, cost=60)
        self.mid_club_4 = MockPlayer.create(element=11, position=3, club=4, cost=70)
        self.mid_club_5 = MockPlayer.create(element=12, position=3, club=5, cost=80)
        # FWDs
        self.fwd_club_3 = MockPlayer.create(element=13, position=4, club=3, cost=60)
        self.fwd_club_4 = MockPlayer.create(element=14, position=4, club=4, cost=75)
        self.fwd_club_5 = MockPlayer.create(element=15, position=4, club=5, cost=80)
        self.fwd_club_1 = MockPlayer.create(element=16, position=4, club=1, cost=45)
        self.fwd_club_2 = MockPlayer.create(element=17, position=4, club=2, cost=55)
        
        self.player_list_feasible = [
            self.gkp_club_1, self.gkp_club_2,
            self.def_club_1, self.def_club_2, self.def_club_3, self.def_club_4, self.def_club_5,
            self.mid_club_1, self.mid_club_2, self.mid_club_3, self.mid_club_4, self.mid_club_5,
            self.fwd_club_3, self.fwd_club_4, self.fwd_club_5
        ]
        
    def test_team(self):
        self.player_list_new = [
            self.gkp_club_1, self.gkp_club_2,
            self.def_club_1, self.def_club_2, self.def_club_3, self.def_club_4, self.def_club_5,
            self.mid_club_1, self.mid_club_2, self.mid_club_3, self.mid_club_4, self.mid_club_5,
            self.fwd_club_3, self.fwd_club_4, self.fwd_club_1
        ]
        
        team_1 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=1)
        team_2 = Team(self.player_list_new, money_in_bank=0, free_transfers=1)
        team_3 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=0)
        team_4 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=1)
        self.assertNotEqual(team_1, team_2, 'They have different players')
        self.assertNotEqual(team_1, team_3, 'They have differnt free transfers')
        self.assertEqual(team_1, team_4, 'They have the same team')
    
    def test_is_feasible(self):
        # too many of one club
        player_list_bad_1 = [
            self.gkp_club_1, self.gkp_club_2,
            self.def_club_1, self.def_club_2, self.def_club_3, self.def_club_4, self.def_club_5,
            self.mid_club_1, self.mid_club_2, self.mid_club_3, self.mid_club_4, self.mid_club_5,
            self.fwd_club_1, self.fwd_club_4, self.fwd_club_5
        ]
        
        # too many fwds
        player_list_bad_2 = [
            self.gkp_club_1, self.gkp_club_2,
            self.def_club_1, self.def_club_2, self.def_club_3, self.def_club_4, self.def_club_5,
            self.mid_club_1, self.mid_club_2, self.mid_club_3, self.mid_club_4, self.mid_club_5,
            self.fwd_club_1, self.fwd_club_4, self.fwd_club_5, self.fwd_club_4
        ]
        
        self.assertTrue(Team(self.player_list_feasible, money_in_bank=0, free_transfers=1).is_feasible())
        self.assertTrue(Team(self.player_list_feasible, money_in_bank=0, free_transfers=-1).is_feasible())
        self.assertTrue(Team(self.player_list_feasible, money_in_bank=1, free_transfers=-1).is_feasible())
        self.assertFalse(Team(self.player_list_feasible, money_in_bank=-1, free_transfers=0).is_feasible())
        
        
        self.assertFalse(Team(player_list_bad_1, money_in_bank=0, free_transfers=1).is_feasible())
        self.assertFalse(Team(player_list_bad_1, money_in_bank=0, free_transfers=-1).is_feasible())
        self.assertFalse(Team(player_list_bad_1, money_in_bank=1, free_transfers=-1).is_feasible())
        self.assertFalse(Team(player_list_bad_1, money_in_bank=-1, free_transfers=0).is_feasible())
        
        self.assertFalse(Team(player_list_bad_2, money_in_bank=0, free_transfers=1).is_feasible())
        self.assertFalse(Team(player_list_bad_2, money_in_bank=0, free_transfers=-1).is_feasible())
        self.assertFalse(Team(player_list_bad_2, money_in_bank=1, free_transfers=-1).is_feasible())
        self.assertFalse(Team(player_list_bad_2, money_in_bank=-1, free_transfers=0).is_feasible())
        
    def test_optimal_formation(self):
        team = Team(self.player_list_feasible, money_in_bank=0, free_transfers=1)
        
        # player map
        pm = {
            1: self.gkp_club_1,
            2: self.gkp_club_2,
            3: self.def_club_1,
            4: self.def_club_2,
            5: self.def_club_3,
            6: self.def_club_4,
            7: self.def_club_5,
            8: self.mid_club_1,
            9: self.mid_club_2,
            10: self.mid_club_3,
            11: self.mid_club_4,
            12: self.mid_club_5,
            13: self.fwd_club_3,
            14: self.fwd_club_4,
            15: self.fwd_club_5
        }
        
        # expected points map
        epm_A = {
            1: 5.0,
            2: 1.0,
            3: 5.0,
            4: 5.0,
            5: 5.0,
            6: 5.0,
            7: 1.0,
            8: 5.0,
            9: 5.0,
            10: 5.0,
            11: 5.0,
            12: 1.0,
            13: 5.0,
            14: 5.0,
            15: 1.0
        }
        
        class MockCalculatorA(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return epm_A[player_id]
            
        result_actual_A = team.optimal_formation(MockCalculatorA(), 0, 0)
        result_expected_A = {
            1: set([(pm[1], epm_A[1])]),
            2: set([(pm[3], epm_A[3]), (pm[4], epm_A[4]), (pm[5], epm_A[5]), (pm[6], epm_A[6])]),
            3: set([(pm[8], epm_A[8]), (pm[9], epm_A[9]), (pm[10], epm_A[10]), (pm[11], epm_A[11])]),
            4: set([(pm[13], epm_A[13]), (pm[14], epm_A[14])])
        }
        self.assertDictEqual(result_actual_A, result_expected_A)
        self.assertEqual(len(result_actual_A[1]), 1)
        self.assertEqual(len(result_actual_A[2]), 4)
        self.assertEqual(len(result_actual_A[3]), 4)
        self.assertEqual(len(result_actual_A[4]), 2)
        self.assertAlmostEqual( sum(sum(y[1] for y in x) for x in result_actual_A.values()) , 55)
        
        # expected points map
        epm_B = {
            1: 1.0,
            2: 2.0,
            3: 3.0,
            4: 4.0,
            5: 5.0,
            6: 6.0,
            7: 7.0,
            8: 8.0,
            9: 9.0,
            10: 10.0,
            11: 11.0,
            12: 12.0,
            13: 13.0,
            14: 14.0,
            15: 15.0
        }
        
        class MockCalculatorB(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return epm_B[player_id]
            
        result_actual_B = team.optimal_formation(MockCalculatorB(), 0, 0)
        result_expected_B = {
            1: set([(pm[2], epm_B[2])]),
            2: set([(pm[5], epm_B[5]), (pm[6], epm_B[6]), (pm[7], epm_B[7])]),
            3: set([(pm[9], epm_B[9]), (pm[10], epm_B[10]), (pm[11], epm_B[11]), (pm[12], epm_B[12])]),
            4: set([(pm[13], epm_B[13]), (pm[14], epm_B[14]), (pm[15], epm_B[15])])
        }
        self.assertDictEqual(result_actual_B, result_expected_B)
        self.assertEqual(len(result_actual_B[1]), 1)
        self.assertEqual(len(result_actual_B[2]), 3)
        self.assertEqual(len(result_actual_B[3]), 4)
        self.assertEqual(len(result_actual_B[4]), 3)
        self.assertAlmostEqual( sum(sum(y[1] for y in x) for x in result_actual_B.values()) , 104)
        
        # expected points map
        epm_C = {
            1: 15.0,
            2: 14.0,
            3: 13.0,
            4: 12.0,
            5: 11.0,
            6: 10.0,
            7: 9.0,
            8: 8.0,
            9: 7.0,
            10: 6.0,
            11: 5.0,
            12: 4.0,
            13: 3.0,
            14: 2.0,
            15: 1.0
        }
        
        class MockCalculatorC(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return epm_C[player_id]
            
        result_actual_C = team.optimal_formation(MockCalculatorC(), 0, 0)
        result_expected_C = {
            1: set([(pm[1], epm_C[1])]),
            2: set([(pm[3], epm_C[3]), (pm[4], epm_C[4]), (pm[5], epm_C[5]), (pm[6], epm_C[6]), (pm[7], epm_C[7])]),
            3: set([(pm[8], epm_C[8]), (pm[9], epm_C[9]), (pm[10], epm_C[10]), (pm[11], epm_C[11])]),
            4: set([(pm[13], epm_C[13])])
        }
        self.assertDictEqual(result_actual_C, result_expected_C)
        self.assertEqual(len(result_actual_C[1]), 1)
        self.assertEqual(len(result_actual_C[2]), 5)
        self.assertEqual(len(result_actual_C[3]), 4)
        self.assertEqual(len(result_actual_C[4]), 1)
        self.assertAlmostEqual( sum(sum(y[1] for y in x) for x in result_actual_C.values()) , 99)
        
        # expected points map
        epm_D = {
            1: 0.0,
            2: 1.0,
            3: 2.0,
            4: 2.0,
            5: 2.0,
            6: 2.0,
            7: 2.0,
            8: 0.0,
            9: 0.0,
            10: 0.0,
            11: 1.0,
            12: 1.0,
            13: 2.0,
            14: 2.0,
            15: 2.0
        }
        
        class MockCalculatorD(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return epm_D[player_id]
            
        result_actual_D = team.optimal_formation(MockCalculatorD(), 0, 0)
        result_expected_D = {
            1: set([(pm[2], epm_D[2])]),
            2: set([(pm[3], epm_D[3]), (pm[4], epm_D[4]), (pm[5], epm_D[5]), (pm[6], epm_D[6]), (pm[7], epm_D[7])]),
            3: set([(pm[11], epm_D[11]), (pm[12], epm_D[12])]),
            4: set([(pm[13], epm_D[13]), (pm[14], epm_D[14]), (pm[15], epm_D[15])])
        }
        self.assertDictEqual(result_actual_D, result_expected_D)
        self.assertEqual(len(result_actual_D[1]), 1)
        self.assertEqual(len(result_actual_D[2]), 5)
        self.assertEqual(len(result_actual_D[3]), 2)
        self.assertEqual(len(result_actual_D[4]), 3)
        self.assertAlmostEqual( sum(sum(y[1] for y in x) for x in result_actual_D.values()) , 19)
        
    def test_score(self):
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                if gameweek == 1:
                    return 10
                elif gameweek == 2:
                    return 20
                elif gameweek == 3:
                    return 30
        
        team_transfers_p2 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=2)
        actual = team_transfers_p2.score(epc=MockCalculator(), gameweek=1, as_of_gameweek=0, horizon=3)
        expected = 12 * (10 + 20 + 30)
        self.assertAlmostEqual(actual, expected, msg='no transfer adjustment should be applied')
        
        team_transfers_p1 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=1)
        actual = team_transfers_p1.score(epc=MockCalculator(), gameweek=1, as_of_gameweek=0, horizon=3)
        expected = 12 * (10 + 20 + 30)
        self.assertAlmostEqual(actual, expected, msg='no transfer adjustment should be applied')
        
        team_transfers_p0 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=0)
        actual = team_transfers_p0.score(epc=MockCalculator(), gameweek=1, as_of_gameweek=0, horizon=3)
        expected = 12 * (10 + 20 + 30) - 2
        self.assertAlmostEqual(actual, expected, msg='penalty should be applied')
        
        team_transfers_m1 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=-1)
        actual = team_transfers_m1.score(epc=MockCalculator(), gameweek=1, as_of_gameweek=0, horizon=3)
        expected = 12 * (10 + 20 + 30) - 4
        self.assertAlmostEqual(actual, expected, msg='penalty should be applied')
        
        team_transfers_m2 = Team(self.player_list_feasible, money_in_bank=0, free_transfers=-2)
        actual = team_transfers_m2.score(epc=MockCalculator(), gameweek=1, as_of_gameweek=0, horizon=3)
        expected = 12 * (10 + 20 + 30) - 8
        self.assertAlmostEqual(actual, expected, msg='penalty should be applied')
        
        actual = team_transfers_m2.score(epc=MockCalculator(), gameweek=1, as_of_gameweek=0, horizon=1)
        expected = 12 * (10) - 8
        self.assertAlmostEqual(actual, expected, msg='penalty should be applied but only one week horizon')
        
        epm_1 = {
            1: 1.0,
            2: 2.0,
            3: 3.0,
            4: 4.0,
            5: 5.0,
            6: 6.0,
            7: 7.0,
            8: 8.0,
            9: 9.0,
            10: 10.0,
            11: 11.0,
            12: 12.0,
            13: 13.0,
            14: 14.0,
            15: 15.0
        }
        
        epm_2 = {
            1: 15.0,
            2: 14.0,
            3: 13.0,
            4: 12.0,
            5: 11.0,
            6: 10.0,
            7: 9.0,
            8: 8.0,
            9: 7.0,
            10: 6.0,
            11: 5.0,
            12: 4.0,
            13: 3.0,
            14: 2.0,
            15: 1.0
        }
        
        class MockCalculatorComplicated(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                if gameweek == 1:
                    return epm_1[player_id]
                elif gameweek == 2:
                    return epm_2[player_id]
                
        actual = team_transfers_p1.score(epc=MockCalculatorComplicated(), gameweek=1, as_of_gameweek=0, horizon=2)
        expected = 104 + 15 + 99 + 15
        self.assertAlmostEqual(actual, expected)
        
    def test_make_transfer(self):
        old_team = Team(self.player_list_feasible, 0, 0)
        
        # substitute a goalkeeper that costs too much
        gkp_club_10 = MockPlayer.create(element=100, position=1, club=10, cost=50)
        new_team = make_transfer(old_team, self.gkp_club_1, gkp_club_10)
        self.assertEqual(new_team.money_in_bank, -10)
        self.assertEqual(new_team.free_transfers, -1)
        self.assertFalse(new_team.is_feasible())
        self.assertTrue(gkp_club_10 in new_team.players[1])
        self.assertTrue(self.gkp_club_1 not in new_team.players[1])
        
        # following the first substitution substitute a fwd which is cheap
        fwd_club_10 = MockPlayer.create(element=120, position=4, club=10, cost=50)
        new_new_team = make_transfer(new_team, self.fwd_club_5, fwd_club_10)
        self.assertEqual(new_new_team.money_in_bank, 20)
        self.assertEqual(new_new_team.free_transfers, -2)
        self.assertTrue(new_new_team.is_feasible())
        self.assertTrue(fwd_club_10 in new_new_team.players[4])
        self.assertTrue(self.fwd_club_5 not in new_new_team.players[4])
        
        # try instead to make another transfer into another club 1 player - should not be feasible as too many players
        another_mid_club_1 = MockPlayer.create(element=130, position=3, club=1, cost=50)
        team_too_many_from_one_club = make_transfer(old_team, self.mid_club_5, another_mid_club_1)
        self.assertEqual(team_too_many_from_one_club.money_in_bank, 30)
        self.assertEqual(team_too_many_from_one_club.free_transfers, -1)
        self.assertFalse(team_too_many_from_one_club.is_feasible())
        self.assertTrue(another_mid_club_1 in team_too_many_from_one_club.players[3])
        self.assertTrue(self.mid_club_5 not in team_too_many_from_one_club.players[3])
        
    def test_optimize_team_1(self):
        # no candidates
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return 0
        
        team = Team(player_list=self.player_list_feasible, money_in_bank=0, free_transfers=1)
        candidates = []
        max_transfers = 1
        epc = MockCalculator()
        gameweek = 1
        as_of_gameweek = 0
        horizon = 1
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon)
        
        self.assertEqual(len(top_three), 1)
        x_score, x_team = heapq.heappop(top_three)
        self.assertEqual(x_score, 0)
        self.assertEqual(x_team.money_in_bank, 0)
        self.assertEqual(x_team.free_transfers, 1)
        
    def test_optimize_team_2(self):
        # cannot fit candidates in as not enough money or too many of one team
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return 0
        
        team = Team(player_list=self.player_list_feasible, money_in_bank=0, free_transfers=1)
        candidates = [
            Player(element=200, name='Salah', position=3, club=20, cost=120),
            Player(element=300, name='Potato man', position=4, club=1, cost=40)
        ]
        max_transfers = 1
        epc = MockCalculator()
        gameweek = 1
        as_of_gameweek = 0
        horizon = 1
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon)
        
        self.assertEqual(len(top_three), 1)
        x_score, x_team = heapq.heappop(top_three)
        self.assertEqual(x_score, 0)
        self.assertEqual(x_team.money_in_bank, 0)
        self.assertEqual(x_team.free_transfers, 1)
        
    def test_optimize_team_3(self):
        # one candidate who doesn't improve the team
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return 0
        
        team = Team(player_list=self.player_list_feasible, money_in_bank=0, free_transfers=1)
        candidates = [
            Player(element=200, name='Candidate who does nothing for you', position=3, club=20, cost=70)
        ]
        max_transfers = 1
        epc = MockCalculator()
        gameweek = 1
        as_of_gameweek = 0
        horizon = 1
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon)
        
        self.assertEqual(len(top_three), 3)
        
        third_best_score, third_best_team = heapq.heappop(top_three)
        self.assertEqual(third_best_score, -2)
        self.assertEqual(third_best_team.money_in_bank, 0)
        self.assertEqual(third_best_team.free_transfers, 0)
        
        second_best_score, second_best_team = heapq.heappop(top_three)
        self.assertEqual(second_best_score, -2)
        self.assertEqual(second_best_team.money_in_bank, 10)
        self.assertEqual(second_best_team.free_transfers, 0)
        
        first_best_score, first_best_team = heapq.heappop(top_three)
        self.assertEqual(first_best_score, 0)
        self.assertEqual(first_best_team.money_in_bank, 0)
        self.assertEqual(first_best_team.free_transfers, 1)
        
    def test_optimize_team_4(self):
        # one candidate who dramatically improves the team and is set as captain
        new_player = Player(element=200, name='Doaky', position=4, club=20, cost=70)
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                return 10 if player_id == 200 else 0
        
        team = Team(player_list=self.player_list_feasible, money_in_bank=0, free_transfers=1)
        candidates = [
            new_player
        ]
        max_transfers = 1
        epc = MockCalculator()
        gameweek = 1
        as_of_gameweek = 0
        horizon = 1
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon)
        
        self.assertEqual(len(top_three), 3)
        
        third_best_score, third_best_team = heapq.heappop(top_three)
        self.assertEqual(third_best_score, 0)
        self.assertEqual(third_best_team.money_in_bank, 0)
        self.assertEqual(third_best_team.free_transfers, 1)
        
        second_best_score, second_best_team = heapq.heappop(top_three)
        self.assertEqual(second_best_score, 2*10-2)
        self.assertEqual(second_best_team.money_in_bank, 5)
        self.assertEqual(second_best_team.free_transfers, 0)
        
        first_best_score, first_best_team = heapq.heappop(top_three)
        self.assertEqual(first_best_score, 2*10-2)
        self.assertEqual(first_best_team.money_in_bank, 10)
        self.assertEqual(first_best_team.free_transfers, 0)
        
    def test_optimize_team_5(self):
        # one candidate who improves the team but not as good as the captain
        new_player = Player(element=200, name='Doaky', position=4, club=20, cost=70)
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                if player_id == 7: return 10
                if player_id == 200: return 3
                return 0
        
        team = Team(player_list=self.player_list_feasible, money_in_bank=0, free_transfers=1)
        candidates = [
            new_player
        ]
        max_transfers = 1
        epc = MockCalculator()
        gameweek = 1
        as_of_gameweek = 0
        horizon = 1
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon)
        
        self.assertEqual(len(top_three), 3)
        
        third_best_score, third_best_team = heapq.heappop(top_three)
        self.assertEqual(third_best_score, 10 + 10)
        self.assertEqual(third_best_team.money_in_bank, 0)
        self.assertEqual(third_best_team.free_transfers, 1)
        
        second_best_score, second_best_team = heapq.heappop(top_three)
        self.assertEqual(second_best_score, 10 + 10 + 3 - 2)
        self.assertEqual(second_best_team.money_in_bank, 5)
        self.assertEqual(second_best_team.free_transfers, 0)
        
        first_best_score, first_best_team = heapq.heappop(top_three)
        self.assertEqual(first_best_score, 10 + 10 + 3 - 2)
        self.assertEqual(first_best_team.money_in_bank, 10)
        self.assertEqual(first_best_team.free_transfers, 0)
        
    def test_optimize_team_6(self):
        # one candidate who is bad for the next game but good over the long run - not as good as capitain
        new_player = Player(element=200, name='Doaky', position=4, club=20, cost=70)
        class MockCalculator(ExpectedPointsCalculator):
            def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
                if player_id == 7: return 10
                if player_id == 200:
                    if gameweek == 1: return 0
                    if gameweek == 2: return 3
                return 0
        
        team = Team(player_list=self.player_list_feasible, money_in_bank=0, free_transfers=1)
        candidates = [
            new_player
        ]
        max_transfers = 1
        epc = MockCalculator()
        gameweek = 1
        as_of_gameweek = 0
        
        # one week horizon
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon=1)
        
        self.assertEqual(len(top_three), 3)
        
        third_best_score, third_best_team = heapq.heappop(top_three)
        self.assertEqual(third_best_score, 10 + 10 - 2)
        self.assertEqual(third_best_team.money_in_bank, 5)
        self.assertEqual(third_best_team.free_transfers, 0)
        
        second_best_score, second_best_team = heapq.heappop(top_three)
        self.assertEqual(second_best_score, 10 + 10 - 2)
        self.assertEqual(second_best_team.money_in_bank, 10)
        self.assertEqual(second_best_team.free_transfers, 0)
        
        first_best_score, first_best_team = heapq.heappop(top_three)
        self.assertEqual(first_best_score, 10 + 10)
        self.assertEqual(first_best_team.money_in_bank, 0)
        self.assertEqual(first_best_team.free_transfers, 1)
        
        # two week horizon
        top_three = optimize_team(team, candidates, max_transfers, epc, gameweek, as_of_gameweek, horizon=2)
        
        self.assertEqual(len(top_three), 3)
        
        third_best_score, third_best_team = heapq.heappop(top_three)
        self.assertEqual(third_best_score, 20 + 20)
        self.assertEqual(third_best_team.money_in_bank, 0)
        self.assertEqual(third_best_team.free_transfers, 1)
        
        second_best_score, second_best_team = heapq.heappop(top_three)
        self.assertEqual(second_best_score, 20 + 20 + 3 - 2)
        self.assertEqual(second_best_team.money_in_bank, 5)
        self.assertEqual(second_best_team.free_transfers, 0)
        
        first_best_score, first_best_team = heapq.heappop(top_three)
        self.assertEqual(first_best_score, 20 + 20 + 3 - 2)
        self.assertEqual(first_best_team.money_in_bank, 10)
        self.assertEqual(first_best_team.free_transfers, 0)
        
    def test_optimize_team_7(self):
        # write tests for two candidates
        pass

In [3]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_is_feasible (__main__.TestTeam.test_is_feasible) ... ok
test_make_transfer (__main__.TestTeam.test_make_transfer) ... ok
test_optimal_formation (__main__.TestTeam.test_optimal_formation) ... ok
test_optimize_team_1 (__main__.TestTeam.test_optimize_team_1) ... ok
test_optimize_team_2 (__main__.TestTeam.test_optimize_team_2) ... ok
test_optimize_team_3 (__main__.TestTeam.test_optimize_team_3) ... ok
test_optimize_team_4 (__main__.TestTeam.test_optimize_team_4) ... ok
test_optimize_team_5 (__main__.TestTeam.test_optimize_team_5) ... ok
test_optimize_team_6 (__main__.TestTeam.test_optimize_team_6) ... ok
test_optimize_team_7 (__main__.TestTeam.test_optimize_team_7) ... ok
test_score (__main__.TestTeam.test_score) ... ok
test_team (__main__.TestTeam.test_team) ... ok

----------------------------------------------------------------------
Ran 12 tests in 1.672s

OK
